### 꿀리뷰
- 꿀리뷰 사이트에서 카테고리별 상품의 제품명, 전체평점, 최저가 데이터를 수집하세요.
- https://www.ggulreview.com/

In [1]:
class Ggulreview:
    
    def __init__(self, category="laptops"):
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        self.driver = webdriver.Chrome(options=options)
        url = "https://www.ggulreview.com/p/{}".format(category)
        self.driver.get(url)
        self.category = category
        self.total_page = 1
        self.df = None
        
    def category_list(self):
        response = requests.get("https://www.ggulreview.com/")
        dom = BeautifulSoup(response.text, "html.parser") 
        elements = dom.select(".active-categories-list__item")
        categories = {}
        for element in elements:
            category = element.select_one("a").get("href").split("/")[-1]
            text = element.select_one("a").text
            categories[text] = category
        return categories
    
    def change_category(self, category):
        url = "https://www.ggulreview.com/p/{}".format(category)
        self.driver.get(url)
        self.category = category    
    
    def __more(self):
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.driver.find_element_by_css_selector(".product-list-view__more").click()
        time.sleep(2)
        self.total_page += 1
    
    def crawling(self):
        
        # 더보기
        self.total_page = 1
        try:
            while True:
                self.__more()
        except:
            pass
        
        elements = self.driver.find_elements_by_css_selector(".product-list-item")
        datas = []
        for element in elements:
            data = {}
            # 제품명 데이터 가져오기
            data["제품명"] = element.find_element_by_css_selector('.product-list-item__desc__name').text
            # 전체 평점
            data["전체평점"] = element.find_element_by_css_selector('.product-list-item__desc__eval__score').text
            # 최저가
            data["최저가"] = element.find_element_by_css_selector('.product-list-item__desc__price').text
            datas.append(data)

        self.df = pd.DataFrame(datas)
        
    def close(self):
        self.driver.quit()

In [2]:
# 객체 생성
gr = Ggulreview()

In [3]:
# 현재 카테고리
gr.category

'laptops'

In [4]:
# 카테고리 리스트 출력

In [5]:
gr.category_list()

{'노트북': 'laptops',
 '무선 청소기': 'cordless-vacuums',
 '로봇 청소기': 'robotic-vacuums',
 '가습기': 'humidifiers',
 'TV': 'televisions',
 '무선 이어폰': 'wireless-earphones',
 '무선 스피커': 'wireless-speakers',
 '믹서기': 'blenders',
 '전기밥솥': 'rice-cookers',
 '에어프라이어': 'air-fryers',
 '공기청정기': 'air-purifiers',
 '건조기': 'dryers',
 '세탁기': 'washing-machines',
 '식기 세척기': 'dishwashers',
 '냉장고': 'refrigerators',
 '빔 프로젝터': 'projectors',
 '모니터': 'monitors',
 '에어컨': 'air-conditioners',
 '선풍기': 'fans',
 '제습기': 'dehumidifiers'}

In [6]:
# 선풍기로 카테고리 변경

In [7]:
gr.change_category("fans")

In [8]:
gr.category

'fans'

In [9]:
# 데이터 수집

In [10]:
gr.crawling()

In [11]:
# 수집된 데이터 확인

In [12]:
gr.df.head()

,제품명,전체평점,최저가
0,발뮤다 그린팬S 무선,4.5,"370,000원"
1,한일전기 아기바람 ABFL-1470RTDC,4.2,"140,000원"
2,한일전기 아기바람 ABFL-1460RDC,4.2,"120,000원"
3,샤오미 스마트미 3세대 무선,4.0,"60,000원"
4,르젠 TR 입체회전,4.2,"70,000원"


In [13]:
# 브라우져 닫기

In [14]:
gr.close()